In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import os
import sys

import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import tarfile
import math



In [ ]:
# function to get the image
def openImage(folder,filename):
	return cv2.imread(os.path.join(folder,filename))

In [ ]:
# Specify the path to the zipped folder in Google Drive
tar_path = '/content/drive/MyDrive/bibimage.tar.gz'

# Specify the destination folder where you want to extract the contents
extract_path = '/content/drive/MyDrive'

# Create the destination folder if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

with tarfile.open(tar_path, 'r:gz') as tar_ref:
    tar_ref.extractall(extract_path)



In [ ]:
# Get the RGB Histograms
def getHistImage(img):
	histBlue = cv2.calcHist([img],[0],None,[16],[0,256])
	histGreen = cv2.calcHist([img],[1],None,[16],[0,256])
	histRed = cv2.calcHist([img],[2],None,[16],[0,256])
	return [histBlue, histGreen, histRed]

In [ ]:
#image1=openImage('/content/drive/MyDrive/BE','01.jpg')

In [ ]:
#getHistImage(image1)

In [ ]:
%cd /content/drive/MyDrive/BE

/content/drive/MyDrive/BE


# **Premier Outil**

**Histogramme de couleur**

In [ ]:
path = "/content/drive/MyDrive/BE"
images = os.listdir(path)
color = ['r', 'g', 'b']
for image_ in images:
    # Load the image
    image = np.array(Image.open(image_))

    # Calculate the histogram

    # Set up the figure and axis
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

    # Display the image
    ax1.imshow(image, )
    ax1.set_title('Image')
    ax1.axis('off')
    for i in range(3):
        hist = cv2.calcHist([image[..., i]], [0], None, [64], [0, 256])
        np.savetxt(f'/content/drive/MyDrive/BE_Histograms/{i}_{image_}.txt', hist.flatten(), fmt='%d')


        # Display the histogram
        ax2.plot(hist, c=color[i])
        ax2.set_title('Histogram')
        ax2.set_xlabel('Pixel Value')
        ax2.set_ylabel('Frequency')

        # Adjust layout for better visualization
    plt.tight_layout()

    # Show the plot
    plt.show()

**Descripteur de texture**

In [ ]:
import cv2
# Charger une image
path = "/content/drive/MyDrive/BE"
images = os.listdir(path)
for img in images:
  image = cv2.imread(f'{path}/{img}')
  results = []

# Convertir l'image en niveaux de gris
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Calculer la Transformée de Fourier 2D
  fourier_transform = np.fft.fft2(gray_image)

# Calculer l'image du spectre en amplitude
  amplitude_spectrum = np.abs(fourier_transform)


# Découper la moitié supérieure en 6x3 blocs
  blocks = np.array_split(amplitude_spectrum[:gray_image.shape[0]//2, :], 6, axis=1)

  subblocks = [np.array_split(block, 3, axis=1) for block in blocks]

    # Flatten the list of subblocks into a 1D list
  flattened_subblocks = [subblock for block in subblocks for subblock in block]



# Calculer le logarithme de l'énergie moyenne sur chacun des blocs
  log_energy_results = []
  for block in flattened_subblocks:
     # print(block)
      energy = np.mean(np.square(np.log(1 + block)))
      log_energy_results.append(energy)
      #np.savetxt(f'/content/drive/MyDrive/BE_Energy/{image}.txt', log_energy_results, fmt='%d')


# Mémoriser les 18 résultats pour chaque image
# (Supposons que vous avez une liste appelée 'results' pour stocker ces résultats)
  results.append(log_energy_results)

  output_file_path = f"/content/drive/MyDrive/BE_Energy/results_{img.split('.')[0]}.txt"
  with open(output_file_path, 'w') as file:
        for result in log_energy_results:
            file.write(f"{result}\n")

# Afficher l'image et le spectre en amplitude pour illustrer le processus
#  fig, axs = plt.subplots(1, 3, figsize=(15, 5))

#  axs[0].imshow(gray_image, cmap='gray')
#  axs[0].set_title('Image en niveaux de gris')

#  axs[1].imshow(np.log(1 + amplitude_spectrum), cmap='gray')
#  axs[1].set_title('Spectre en amplitude (log)')

#  axs[2].bar(range(len(log_energy_results)), log_energy_results, color='blue')
#  axs[2].set_title('Logarithme de l\'énergie moyenne sur chaque bloc')

#  plt.show()

**Calculer descripteur de l'image en entree**

In [ ]:
def calculate_texture_descriptor(image):

  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  fourier_transform = np.fft.fft2(gray_image)

  amplitude_spectrum = np.abs(fourier_transform)

  blocks = np.array_split(amplitude_spectrum[:gray_image.shape[0]//2, :], 6, axis=1)

  subblocks = [np.array_split(block, 3, axis=1) for block in blocks]

  flattened_subblocks = [subblock for block in subblocks for subblock in block]

  log_energy_results = []
  for block in flattened_subblocks:
      energy = np.mean(np.square(np.log(1 + block)))
      log_energy_results.append(energy)

  return log_energy_results




In [ ]:
def calculate_histogram(image):
  for i in range(3):
        hist = cv2.calcHist([image[..., i]], [0], None, [64], [0, 256])
  return hist




**distance entre histogramme**

In [ ]:

def euclidean_distance(hist1, hist2):

    return np.linalg.norm(hist1 - hist2)


**distance entre descripteurs de texture**

In [ ]:
def manhattan_distance(list1, list2):
    """
    Calcule la distance de Manhattan entre deux listes.
    """
    if len(list1) != len(list2):
        raise ValueError("Les listes doivent avoir la même longueur.")
    return sum(abs(a - b) for a, b in zip(list1, list2))



**Comparaison entre image en entree et le reste des images**

In [ ]:

# Load the input image
input_image_path = "/content/drive/MyDrive/BE/20.jpg"
input_image = cv2.imread(input_image_path)

# Calculate histogram and texture descriptor for the input image
input_histogram = calculate_histogram(input_image)
input_texture_descriptor = calculate_texture_descriptor(input_image)

# Folder containing other BE images
be_folder_path = "/content/drive/MyDrive/BE"
be_images = os.listdir(be_folder_path)

# Lists to store distances with image names
distances_combined = []
distance_histogram = []
distance_texture = []

# Calculate distances for each image
for be_image_name in be_images:
    # Load the image
    be_image = cv2.imread(os.path.join(be_folder_path, be_image_name))

    # Calculate histogram and texture descriptor for the current BE image
    be_histogram = calculate_histogram(be_image)
    be_texture_descriptor = calculate_texture_descriptor(be_image)

    # Calculate distances
    distance_histogram = euclidean_distance(input_histogram, be_histogram)
    distance_texture = manhattan_distance(input_texture_descriptor,be_texture_descriptor)


    # Combine distances using a weighted average
    weight_histogram = 0.7  # Adjust weights as needed
    weight_texture = 0.3
    combined_distance = (weight_histogram * distance_histogram) + (weight_texture * distance_texture)

    distances_combined.append((be_image_name, abs(combined_distance)))

# Sort distances by increasing order
distances_combined.sort(key=lambda x: x[1])
print("Sorted distances:", distances_combined)  # Add this line for debugging

# Display the top 3 images based on combined distances
print("Top 3 images based on combined distances:")
for img, dist in distances_combined[1:5]:
    print(f"Image: {os.path.join(be_folder_path, img)}, Combined Distance: {dist}")




Sorted distances: [('20.jpg', 0.0), ('32.jpg', 1254.7052643555367), ('35.jpg', 1670.3390696785857), ('33.jpg', 1810.1263256867912), ('12.jpg', 1886.2612230033585), ('29.jpg', 1928.6517344842277), ('23.jpg', 1997.1184083989258), ('39.jpg', 2004.911727966793), ('50.jpg', 2232.7871396961946), ('04.jpg', 2511.9114345906646), ('34.jpg', 2520.6930637360138), ('07.jpg', 2590.9763782552222), ('24.jpg', 2780.341046900314), ('48.jpg', 2819.349581283486), ('15.jpg', 2851.3441546883705), ('49.jpg', 2966.3949010490037), ('40.jpg', 3009.953857427148), ('14.jpg', 3021.843601787045), ('28.jpg', 3090.664528406378), ('10.jpg', 3105.999048940679), ('11.jpg', 3163.5878441324), ('19.jpg', 3221.3599977416484), ('02.jpg', 3229.5056713377894), ('26.jpg', 3265.728332685079), ('41.jpg', 3409.543933328629), ('05.jpg', 3435.7814142478337), ('06.jpg', 3498.035514963075), ('25.jpg', 3553.0496327124833), ('13.jpg', 3638.620673816309), ('36.jpg', 3720.9163794740452), ('22.jpg', 3757.0523091473956), ('42.jpg', 4272.82